This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to use Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json


In [10]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

#uses the requests module to bring in the file and  then uses the .json() to read it as a json file
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2014-01-02&end_date=2014-01-02&api_key=gCCLj7uFdS59KkviLo6B')
r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2014-01-02',
  'end_date': '2014-01-02',
  'frequency': 'daily',
  'data': [['2014-01-02',
    24.24,
    24.76,
    23.77,
    24.18,
    None,
    55735.0,
    1355658.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [9]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [93]:
#question 1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
#(keep in mind that the date format is YYYY-MM-DD)

# this brings in the dataset for the entire year from 2017-01-02 through 2017-12-29 - the market is not open on New Year's
# Day or the 12-30 or 12-31. Thus, there is no data for these dates in the dataset although I called them in my urlstring
r2 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=API_KEY')
json_file = r2.json()
#json_file

In [7]:
#Question 2: Convert the returned JSON object into a Python dictionary.

#This converts the json object into a dictionary
json_file = dict(json_file)
type(json_file)

dict

In [92]:
json_data = json_file['dataset_data'] # goes down a level in the dictionary so that I can call the nested keys
json_data1 = json_data['data'] # all of the data for the dataset
json_columns = json_data['column_names'] # all of the column names of the dataset

# this for loop goes through and places each column name as a dictionary key that can be called with their corresponding data
#points for each day. Thus, I can call all of the opening prices for every single day of 2017 with the key 'Open'
#I will be using data_dict (the dataframe I placed everything into) for the following questions
data_dict = {} #empty dictionary
for i in range(255):
    lval = json_data1[i]
    for b in range(11):
        key = json_columns[b]
        value = lval[b]
        if i < 1:
            data_dict[key] = [lval[b]]
        else:
            data_dict[key].append(lval[b])

In [65]:
#Question 3: Calculate what the highest and lowest opening prices were for the stock in this period.


openprices = [] 
prices = data_dict['Open']

#this for loop checks for any missing data and removes the nones from the list since there were 2 days that were missing data
# and causing an error when attempting to use the max and min functions
for val in prices:
    if val != None:
        openprices.append(val)

print(max(openprices))
print(min(openprices))


53.11
34.0


In [79]:
# Question 4: What was the largest change in any one day (based on High and Low price)?

highprice = data_dict['High']# saves list of High prices
lowprice = data_dict['Low']
maxchange = 0 # empty variable to be filled by for loop
# this for loop compares the high and low prices for each day and then saves them to the maxchange variable if it finds that 
# the change is greater in that comparison than any other previous comparison
for i in range(255):
    h = highprice[i]
    l = lowprice[i]
    change = abs(h - l)
    if change > maxchange:
        maxchange = change
maxchange
    
    


2.8100000000000023

In [82]:
#Question 5: What was the largest change between any two days (based on Closing Price)?

#basically the same code as the previous cell except using two days rather than high and low
closingprice = data_dict['Close']
largest = 0
for i in range(255):
    if i <= 253:
        dayone = closingprice[i]
        daytwo = closingprice[i+1]
        change = abs(dayone - daytwo)
        if change > largest:
            largest = change
largest
    

2.559999999999995

In [87]:
# Question 6: What was the average daily trading volume during this year?

# finds the average trading volume by adding all volumes in the list together and dividing by the length of the list
volume = data_dict['Traded Volume']
sum(volume)/len(volume)


89124.33725490196

In [91]:
# Question 7: What was the median trading volume during this year. 
#     (Note: you may need to implement your own function for calculating the median.)

#finds the median by sorting the volumes from least to greatest then finds the middle which is at index 128 since half of 
#255 is 127.5 thus there are 127 values before the value at index 128 and 127 values after thus making index 128 the median
volume.sort()
median = volume[128]
median


76600.0